In [31]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('invoice.pdf')

docs = loader.load()
print(docs)
print(len(docs))

[Document(page_content='This is a system generated invoice and hence does not require a signature & Stamp\n1\nTotal AED12,075.00\nBalance Due AED12,075.00\nInvoice Date : 23 Jan 2026\nTerms : Due on Receipt\nDue Date : 23 Jan 2026\nVAT No. : 104178814000003\nBytoons Innovations (FZC)\nSharjah \nUnited Arab Emirates\nTRN 104799101100003\ndeepak@bytoons.com\nTAX INVOICE\n# INV-000104\nBalance Due\nAED12,075.00\nBill To \nHOME SEEKERS REAL ESTATE - SOLE PROPRIETORSHIP L.L.C\n Dubai\nUnited Arab Emirates\nTRN 104178814000003\n# Item & Description Qty Rate Taxable \nAmount Tax Amount\n1 Marketing Budget 1.00 10,000.00 10,000.00 500.00\n5.00%\n10,500.00\n2 Marketing Service Fee 1.00 1,500.00 1,500.00 75.00\n5.00%\n1,575.00\nSub Total 11,500.00 575.00 12,075.00\nTax Summary\nTax Details Taxable Amount (AED) Tax Amount (AED)\nStandard Rate (5%) 11,500.00 575.00\nTotal AED11,500.00 AED575.00\nBank Details :\nAccount holder: Bytoons Innovations FZC\nAccount number: 9854887748\nIBAN: AE9608600000

In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
result = model.invoke("What is the capital of India")
 
print(result.content)

The capital of India is **New Delhi**.


In [15]:
ocr_text = ocr_read_document.run("invoice.png")
print("Raw OCR Output:\n----------------\n", ocr_text)

Raw OCR Output:
----------------
 Error reading image: tesseract is not installed or it's not in your PATH. See README file for more information.


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()

from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

# Combine all pages into one text (if multiple pages)
invoice_text = "\n".join([doc.page_content for doc in docs])

# Define a prompt to extract relevant invoice information
prompt_template = """
You are an expert at reading invoices.

Extract the following information from the text below:

- Invoice Number
- Invoice Date
- Vendor / Supplier Name
- Total Amount
- Line Items (Item Name, Quantity, Price per unit, Total Price)

Invoice Text:
{invoice_text}

Provide the output in JSON format with keys: invoice_number, invoice_date, vendor, total_amount, line_items
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["invoice_text"]
)

chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain
result = chain.run(invoice_text=invoice_text)

print(result)


In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json

# Step 1: Load PDF
loader = PyPDFLoader('invoice.pdf')
docs = loader.load()

# Combine all pages into one text string
invoice_text = "\n".join([doc.page_content for doc in docs])

# Step 2: Define a prompt to extract invoice info
prompt_template = """
You are an expert at reading invoices.

Extract the following information from the text below:

- Invoice Number
- Invoice Date
- Vendor / Supplier Name
- Vendor TRN / VAT
- Bill To (Customer Name, Address, TRN)
- Total Amount
- Balance Due
- Line Items (Item Name, Quantity, Rate, Taxable Amount, Tax Amount, Total)
- Sub Total
- Tax Summary (Taxable Amount, Tax Amount, Total, Tax Rate)
- Bank Details (Account Holder, Account Number, IBAN, BIC)
- Business Address

Invoice Text:
{invoice_text}

Provide the output in **JSON format only** with keys exactly like:
invoice_number, invoice_date, vendor, bill_to, total_amount, balance_due, line_items, sub_total, tax_summary, bank_details, business_address
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["invoice_text"],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

# Step 3: Initialize LLM
model = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

parser = JsonOutputParser()

# Step 4: Create a chain
chain = prompt | model | parser

# Step 5: Run chain to extract invoice info
result_json = chain.invoke({'invoice_text' : invoice_text})

print(result_json)



{'invoice_number': 'INV-000104', 'invoice_date': '23 Jan 2026', 'vendor': {'name': 'Bytoons Innovations (FZC)', 'address': 'Sharjah United Arab Emirates', 'trn_vat': '104799101100003', 'email': 'deepak@bytoons.com'}, 'bill_to': {'customer_name': 'HOME SEEKERS REAL ESTATE - SOLE PROPRIETORSHIP L.L.C', 'address': 'Dubai United Arab Emirates', 'trn': '104178814000003'}, 'total_amount': '12075.00', 'balance_due': '12075.00', 'line_items': [{'item_name': 'Marketing Budget', 'quantity': '1.00', 'rate': '10000.00', 'taxable_amount': '10000.00', 'tax_amount': '500.00', 'total': '10500.00'}, {'item_name': 'Marketing Service Fee', 'quantity': '1.00', 'rate': '1500.00', 'taxable_amount': '1500.00', 'tax_amount': '75.00', 'total': '1575.00'}], 'sub_total': {'taxable_amount': '11500.00', 'tax_amount': '575.00', 'total': '12075.00'}, 'tax_summary': [{'tax_rate': '5.00%', 'taxable_amount': '11500.00', 'tax_amount': '575.00', 'total': '12075.00'}], 'bank_details': {'account_holder': 'Bytoons Innovatio

In [35]:
import langchain
print(langchain.__version__)


1.2.7
